## 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# 블로그 데이터 불러오기
# file_path = '/content/Shared drives/SKN09-5th-4team/data/row data/10_NEW_fitpet_articles.csv'
file_path = './data/4_blog_posts.csv'
df = pd.read_csv(file_path)

# 데이터 확인
print(df.head())
print(df.columns)
df.describe()

## 데이터 전처리

In [ ]:
import re
import pandas as pd

In [ ]:
import re
import pandas as pd

# 1. 동물병원 이름 삭제
def remove_specific_phrases(text):
    if pd.isnull(text):
        return ""
    patterns = [
        r'강남\s*동물\s*병원\s*추천',
        r'강남\s*동물\s*병원',
    ]
    for pattern in patterns:
        text = re.sub(pattern, '', text)
    return text

# 2. 해시태그 삭제
def remove_hashtags(text):
    if pd.isnull(text):
        return ""
    text = re.sub(r'#\S+', '', text)
    return text

# 3. 줄바꿈 제거
def remove_newlines(text):
    if pd.isnull(text):
        return ""
    return re.sub(r'[\r\n]+', ' ', text)

# 4. 문장 분리
def split_sentences(text):
    if pd.isnull(text):
        return []
    sentences = re.split(r'(?<=[\.?!~])\s*', text)
    sentences = [s.strip() for s in sentences if s.strip()]
    return sentences

# 5. 광고/홍보 문장 제거
def remove_ad_sentences(sentences):
    ad_keywords = [
        "TV동물농장", "최영민", "동물병원", "진료문의", "오시는 길",
        "이 블로그", "NAVER Corp", "서울특별시"
    ]
    clean_sentences = []
    for sentence in sentences:
        if not any(keyword in sentence for keyword in ad_keywords):
            clean_sentences.append(sentence)
    return clean_sentences

# ✨ 전체 전처리 파이프라인
def preprocess_text(text):
    text = remove_specific_phrases(text)
    text = remove_hashtags(text)
    text = remove_newlines(text)
    sentences = split_sentences(text)
    clean_sentences = remove_ad_sentences(sentences)
    # 다시 문장 이어붙이기 (공백으로)
    cleaned_text = ' '.join(clean_sentences)
    return cleaned_text

# 적용
df['content_clean'] = df['content'].apply(preprocess_text)


# 확인용
print(df.loc[10, 'content'])
print("-" * 50)
print(df.loc[10, 'content_clean'])

In [ ]:
# 6. 특수문자 제거 (한글, 영어, 숫자만 남기기)
def clean_special_characters(text):
    if pd.isnull(text):
        return ""
    # 가-힣, a-zA-Z, 0-9, 공백, .?! 만 남김
    text = re.sub(r'[^가-힣a-zA-Z0-9\s\.?!%]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['content_clean'] = df['content_clean'].apply(clean_special_characters)

In [ ]:
# 확인용
print(df.loc[10, 'content'])
print("-" * 50)
print(df.loc[10, 'content_clean'])

타이틀 전처리

In [ ]:
# 확인용
df['title'].head(10)

In [ ]:
def clean_title(text):
    # "강남" + (공백 0개 이상) + "동물병원" + (공백 0개 이상) + "추천" 제거
    text = re.sub(r'강남\s*동물\s*병원\s*추천', '', text)
    # "강남" + (공백 0개 이상) + "동물병원" 만 있는 것도 제거
    text = re.sub(r'강남\s*동물\s*병원', '', text)

    # 제거 후 남은 중복 공백 정리
    text = re.sub(r'\s+', ' ', text).strip()

    return text

df['title'] = df['title'].apply(clean_title)

In [ ]:
# 확인용
df['title'].head(10)

### 파일 저장

In [ ]:
# 원본 df에서 필요한 컬럼만 복사
df_final = df[['title', 'image', 'URL']].copy()

# content_clean을 content로 추가
df_final['content'] = df['content_clean']

# CSV로 저장
df_final.to_csv('./clean data/4_clean_blog_data.csv', index=False, encoding='utf-8-sig')